# ReEDS Scenarios on PV ICE Tool

To explore different scenarios for furture installation projections of PV (or any technology), ReEDS output data can be useful in providing standard scenarios. ReEDS installation projections are used in this journal as input data to the PV ICE tool. 

Current sections include:

<ol>
    <li> ### Reading a standard ReEDS output file </li>
    <li> ### Saving PCA data as PV ICE input format </li>
    <li> ### Saving State data as PV ICE input format </li>
</ol>


In [75]:
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
plt.rcParams['figure.figsize'] = (12, 8)

In [76]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent.parent / 'PV_ICE' / 'TEMP')

print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\hmirletz\Documents\GitHub\PV_ICE\PV_ICE\TEMP


## Reading a simple standard ReEDS output file
This reEDS output file is the cumulative capacity every other year. To get GW/yr installed, we need to take the difference between the years, then divide by 2.

Not all ReEDS output files seem to work like this, but this code is for munging cumulative capacity files.

In [77]:
#We need to come up with a default location to save ReEDS output files
#reedsFile = str(Path().resolve().parent.parent.parent.parent / 'December Core Scenarios ReEDS Outputs Solar Futures v3a.xlsx')
#print ("Input file is stored in %s" % reedsFile)
reedsFile = str(Path().resolve().parent.parent.parent/ 'PV_ICE' / 'baselines' / 'SupportingMaterial' / '100REby2035-ReEDS.csv')
print ("Input file is stored in %s" % reedsFile)

Input file is stored in C:\Users\hmirletz\Documents\GitHub\PV_ICE\PV_ICE\baselines\SupportingMaterial\100REby2035-ReEDS.csv


In [78]:
REEDSInput = pd.read_csv(reedsFile)
REEDSInput.head

<bound method NDFrame.head of       scenario  year   PCA  Capacity (GW)
0     a1_RE100  2010    p1       0.000016
1     a1_RE100  2010   p10       0.486023
2     a1_RE100  2010  p100       0.000000
3     a1_RE100  2010  p101       0.021315
4     a1_RE100  2010  p102       0.014625
...        ...   ...   ...            ...
8437    a1_Ref  2050   p95      38.563516
8438    a1_Ref  2050   p96      16.298854
8439    a1_Ref  2050   p97      18.744921
8440    a1_Ref  2050   p98       6.378485
8441    a1_Ref  2050   p99      28.449829

[8442 rows x 4 columns]>

# Aggregate down to simple MW Installed input

In [79]:
rawdf = REEDSInput.copy()

#currently, we're ignoring region, so drop PCA
rawdf.drop(columns=['PCA'], inplace=True)

#aggregate and sum by scenarios and year to get an annual (bi-annual) installation by scenario
df = rawdf.groupby(['scenario', 'year'])['Capacity (GW)'].sum()
df_evens = pd.DataFrame(df)
df_evens_byscen = df.unstack(level='scenario')#this df contains all the scenarios, even years only
print(df_evens_byscen)

scenario     a1_RE100      a1_RE95      a1_Ref
year                                          
2010         1.246592     1.246592    1.246592
2012         6.242318     6.242318    6.242318
2014        16.090783    16.090783   16.090783
2016        34.276154    34.276154   34.276154
2018        52.093001    52.093001   52.093001
2020        83.778187    83.778187   83.533329
2022       146.553694   146.553694  102.993795
2024       191.078858   191.078858  128.602753
2026       326.268392   326.268392  177.112068
2028       431.634770   431.634770  230.392738
2030       580.819921   580.819921  309.015319
2032       702.410947   702.410947  333.157319
2034       812.257622   812.257622  354.101299
2036       934.894845   934.894845  385.447237
2038      1078.186255  1078.186255  417.757467
2040      1237.253347  1237.253347  464.569087
2042      1283.271124  1283.384991  512.675339
2044      1320.858174  1322.752809  569.486607
2046      1363.860206  1374.601619  614.658599
2048      141

## Take the difference and Divide by 2
The file provided is cumulative installs, therefore we need to take the difference between years

In [80]:
#take the difference between years to get the annual installs (not cumulative)
#grab scenario names
SCEN = df_evens.index.levels[0]
#len(SCEN)
#create a for loop to iterate through all the provided scenarios
for i in range(len(SCEN)):
    df_evens_byscen[SCEN[i]+'_added_cap(GW)'] = df[SCEN[i]].diff() #takes diff between rows and creates new column
#print(df_evens_byscen)
df_addedCap_evens = df_evens_byscen.filter(regex = 'added_cap') #create df of just added capacity, removes multiIndex
print(df_addedCap_evens)

scenario  a1_RE100_added_cap(GW)  a1_RE95_added_cap(GW)  a1_Ref_added_cap(GW)
year                                                                         
2010                         NaN                    NaN                   NaN
2012                    4.995726               4.995726              4.995726
2014                    9.848465               9.848465              9.848465
2016                   18.185371              18.185371             18.185371
2018                   17.816847              17.816847             17.816847
2020                   31.685186              31.685186             31.440328
2022                   62.775507              62.775507             19.460466
2024                   44.525164              44.525164             25.608958
2026                  135.189534             135.189534             48.509315
2028                  105.366378             105.366378             53.280670
2030                  149.185151             149.185151         

Because we prefer to use real world data whereever possible, anything prior to the current year will use real world installs. Therefore, we can ignore the NaN values in 2010.

Now we divide this added installations in half.

In [81]:
df_annualAdds_evens = df_addedCap_evens/2
print(df_annualAdds_evens)

scenario  a1_RE100_added_cap(GW)  a1_RE95_added_cap(GW)  a1_Ref_added_cap(GW)
year                                                                         
2010                         NaN                    NaN                   NaN
2012                    2.497863               2.497863              2.497863
2014                    4.924232               4.924232              4.924232
2016                    9.092685               9.092685              9.092685
2018                    8.908424               8.908424              8.908424
2020                   15.842593              15.842593             15.720164
2022                   31.387753              31.387753              9.730233
2024                   22.262582              22.262582             12.804479
2026                   67.594767              67.594767             24.254658
2028                   52.683189              52.683189             26.640335
2030                   74.592576              74.592576         

## Create the odd years

In [82]:
#Now create the odd years by duplicating the even years and changing the index
df_odds = df_annualAdds_evens.copy()
df_odds.index = df_odds.index-1 #set the index = one year less
print(df_odds)

scenario  a1_RE100_added_cap(GW)  a1_RE95_added_cap(GW)  a1_Ref_added_cap(GW)
year                                                                         
2009                         NaN                    NaN                   NaN
2011                    2.497863               2.497863              2.497863
2013                    4.924232               4.924232              4.924232
2015                    9.092685               9.092685              9.092685
2017                    8.908424               8.908424              8.908424
2019                   15.842593              15.842593             15.720164
2021                   31.387753              31.387753              9.730233
2023                   22.262582              22.262582             12.804479
2025                   67.594767              67.594767             24.254658
2027                   52.683189              52.683189             26.640335
2029                   74.592576              74.592576         

In [83]:
#put the evens and odds together, sort by year
df_allyrs = pd.concat([df_annualAdds_evens, df_odds])
df_allyrs.sort_index(axis=0, inplace=True)
#df_allyrs_byscen = df_allyrs.unstack(level=0)
#print(df_allyrs_byscen)

#modify by DC:AC ratio(1.2 avg) and 85% average marketshare of c-Si technology and convert to MW
df_cSi_installs = df_allyrs*1.2*0.85*1000
print(df_cSi_installs)

scenario  a1_RE100_added_cap(GW)  a1_RE95_added_cap(GW)  a1_Ref_added_cap(GW)
year                                                                         
2009                         NaN                    NaN                   NaN
2010                         NaN                    NaN                   NaN
2011                 2547.820449            2547.820449           2547.820449
2012                 2547.820449            2547.820449           2547.820449
2013                 5022.716968            5022.716968           5022.716968
2014                 5022.716968            5022.716968           5022.716968
2015                 9274.539118            9274.539118           9274.539118
2016                 9274.539118            9274.539118           9274.539118
2017                 9086.592100            9086.592100           9086.592100
2018                 9086.592100            9086.592100           9086.592100
2019                16159.444770           16159.444770         

In [84]:
#output the file
df_cSi_installs.to_csv('output_reeds4PVICE.csv', index=True)

# Append to Projections Options File

## Save Input Files by PCA

#### Create a copy of the REEDS Input and modify structure for PCA focus

#### Loading Module Baseline. Will be used later to populate all the columsn otehr than 'new_Installed_Capacity_[MW]' which will be supplied by the REEDS model

In [6]:
import PV_ICE
r1 = PV_ICE.Simulation(name='Simulation1', path=testfolder)
r1.createScenario(name='US', file=r'..\baselines\SolarFutures_2021\baseline_modules_US_Reeds.csv')
baseline = r1.scenario['US'].data
baseline = baseline.drop(columns=['new_Installed_Capacity_[MW]'])
baseline.set_index('year', inplace=True)
baseline.index = pd.PeriodIndex(baseline.index, freq='A')  # A -- Annual
baseline.head()

path = C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP


,mod_eff,mod_reliability_t50,mod_reliability_t90,mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,mod_Repair,mod_MerchantTail,mod_Reuse
year,,,,,,,,,,,
2010,14.7,25.0,30.0,0.3,22.7,98.0,0.0,0.0,0.0,0.0,0.0
2011,15.1,25.0,30.0,0.3,23.0,98.0,0.0,0.0,0.0,0.0,0.0
2012,15.4,25.0,30.0,0.3,23.5,98.0,0.0,0.0,0.0,0.0,0.0
2013,16.0,28.0,33.0,0.3,24.2,98.0,0.0,0.0,0.0,0.0,0.0
2014,16.3,28.0,33.0,0.3,26.0,98.0,0.0,0.0,0.0,0.0,0.0


#### For each Scenario and for each PCA, combine with baseline and save as input file

In [8]:
for ii in range (len(rawdf.unstack(level=1))):
    PCA = rawdf.unstack(level=1).iloc[ii].name[1]
    SCEN = rawdf.unstack(level=1).iloc[ii].name[0]
    SCEN=SCEN.replace('+', '_')
    filetitle = SCEN+'_'+PCA +'.csv'
    subtestfolder = os.path.join(testfolder, 'PCAs')
    if not os.path.exists(subtestfolder):
        os.makedirs(subtestfolder)
    filetitle = os.path.join(subtestfolder, filetitle)
    A = rawdf.unstack(level=1).iloc[ii]
    A = A.droplevel(level=0)
    A.name = 'new_Installed_Capacity_[MW]'
    A = pd.DataFrame(A)
    A.index=pd.PeriodIndex(A.index, freq='A')
    A = pd.DataFrame(A)
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 0.85
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 1000   # ReEDS file is in GW.
    # Add other columns
    A = pd.concat([A, baseline.reindex(A.index)], axis=1)
   
    header = "year,new_Installed_Capacity_[MW],mod_eff,mod_reliability_t50,mod_reliability_t90,"\
    "mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,"\
    "mod_Repair,mod_MerchantTail,mod_Reuse\n"\
    "year,MW,%,years,years,%,years,%,%,%,%,%,%\n"

    with open(filetitle, 'w', newline='') as ict:
    # Write the header lines, including the index variable for
    # the last one if you're letting Pandas produce that for you.
    # (see above).
        for line in header:
            ict.write(line)

        #    savedata.to_csv(ict, index=False)
        A.to_csv(ict, header=False)

## Save Input Files By States

#### Reassign data from REEDS Input, as we need one of the columns we dropped.

In [9]:
rawdf = REEDSInput.copy()
#rawdf.drop(columns=['State'], inplace=True)
rawdf.drop(columns=['Tech'], inplace=True)
rawdf.set_index(['Scenario','Year','PCA', 'State'], inplace=True)
rawdf.head(21)


Capacity (GW)
Scenario     Year PCA State               
95-by-35.Adv 2010 p1  WA          0.000017
             2011 p1  WA          0.005158
             2012 p1  WA          0.005158
             2013 p1  WA          0.007146
             2014 p1  WA          0.007146
             2015 p1  WA          0.018253
             2016 p1  WA          0.018253
             2017 p1  WA          0.019460
             2018 p1  WA          0.019460
             2019 p1  WA          0.036320
             2020 p1  WA          0.036320
             2021 p1  WA          0.002434
             2022 p1  WA          0.002434
             2023 p1  WA          0.010066
             2024 p1  WA          0.010066
             2025 p1  WA          0.049445
             2026 p1  WA          0.049445
             2027 p1  WA          0.051342
             2028 p1  WA          0.051342
             2029 p1  WA          0.105931
             2030 p1  WA          0.105931

#### Group data so we can work with the States instead

In [10]:
#df = rawdf.groupby(['Scenario','State', 'Year'])['Capacity (GW)'].sum(axis=0)
df = rawdf.groupby(['Scenario','State', 'Year'])['Capacity (GW)'].sum()
df = pd.DataFrame(df)
df.head()

Capacity (GW)
Scenario          State Year               
95-by-35+Elec.Adv AL    2010       0.000000
                        2011       0.000000
                        2012       0.000000
                        2013       0.000951
                        2014       0.000951

#### For each Scenario and for each STATE, combine with baseline and save as input file

In [11]:
for ii in range (len(df.unstack(level=2))):   
    STATE = df.unstack(level=2).iloc[ii].name[1]
    SCEN = df.unstack(level=2).iloc[ii].name[0]
    SCEN=SCEN.replace('+', '_')
    filetitle = SCEN+'_'+STATE +'.csv'
    
    subtestfolder = os.path.join(testfolder, 'STATEs')
    if not os.path.exists(subtestfolder):
        os.makedirs(subtestfolder)
    filetitle = os.path.join(subtestfolder, filetitle)

    A = df.unstack(level=2).iloc[ii]
    A = A.droplevel(level=0)
    A.name = 'new_Installed_Capacity_[MW]'
    A = pd.DataFrame(A)
    A.index=pd.PeriodIndex(A.index, freq='A')
    A = pd.DataFrame(A)
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 0.85 # marketshares['Si']
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 1000   # ReEDS file is in GW.
    # Add other columns
    A = pd.concat([A, baseline.reindex(A.index)], axis=1)
    
    
    header = "year,new_Installed_Capacity_[MW],mod_eff,mod_reliability_t50,mod_reliability_t90,"\
    "mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,"\
    "mod_Repair,mod_MerchantTail,mod_Reuse\n"\
    "year,MW,%,years,years,%,years,%,%,%,%,%,%\n"

    with open(filetitle, 'w', newline='') as ict:
    # Write the header lines, including the index variable for
    # the last one if you're letting Pandas produce that for you.
    # (see above).
        for line in header:
            ict.write(line)

        #    savedata.to_csv(ict, index=False)
        A.to_csv(ict, header=False)

# Saving US Baseline

### Create a copy of the REEDS Input and modify structure for PCA focus

In [12]:
rawdf = REEDSInput.copy()
#rawdf.drop(columns=['State'], inplace=True)
rawdf.drop(columns=['Tech'], inplace=True)
rawdf.set_index(['Scenario','Year'], inplace=True)
rawdf.head(21)

PCA State  Capacity (GW)
Scenario     Year                         
95-by-35.Adv 2010  p1    WA       0.000017
             2011  p1    WA       0.005158
             2012  p1    WA       0.005158
             2013  p1    WA       0.007146
             2014  p1    WA       0.007146
             2015  p1    WA       0.018253
             2016  p1    WA       0.018253
             2017  p1    WA       0.019460
             2018  p1    WA       0.019460
             2019  p1    WA       0.036320
             2020  p1    WA       0.036320
             2021  p1    WA       0.002434
             2022  p1    WA       0.002434
             2023  p1    WA       0.010066
             2024  p1    WA       0.010066
             2025  p1    WA       0.049445
             2026  p1    WA       0.049445
             2027  p1    WA       0.051342
             2028  p1    WA       0.051342
             2029  p1    WA       0.105931
             2030  p1    WA       0.105931

In [13]:
#df = rawdf.groupby(['Scenario','Year'])['Capacity (GW)'].sum(axis=0)
df = rawdf.groupby(['Scenario','Year'])['Capacity (GW)'].sum()


### Loading Module Baseline. Will be used later to populate all the columsn other than 'new_Installed_Capacity_[MW]' which will be supplied by the REEDS model

In [14]:
import PV_ICE
r1 = PV_ICE.Simulation(name='Simulation1', path=testfolder)
r1.createScenario(name='US', file=r'..\baselines\SolarFutures_2021\baseline_modules_US_Reeds.csv')
baseline = r1.scenario['US'].data
baseline = baseline.drop(columns=['new_Installed_Capacity_[MW]'])
baseline.set_index('year', inplace=True)
baseline.index = pd.PeriodIndex(baseline.index, freq='A')  # A -- Annual
baseline.head()

path = C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP


,mod_eff,mod_reliability_t50,mod_reliability_t90,mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,mod_Repair,mod_MerchantTail,mod_Reuse
year,,,,,,,,,,,
2010,14.7,25.0,30.0,0.3,22.7,98.0,0.0,0.0,0.0,0.0,0.0
2011,15.1,25.0,30.0,0.3,23.0,98.0,0.0,0.0,0.0,0.0,0.0
2012,15.4,25.0,30.0,0.3,23.5,98.0,0.0,0.0,0.0,0.0,0.0
2013,16.0,28.0,33.0,0.3,24.2,98.0,0.0,0.0,0.0,0.0,0.0
2014,16.3,28.0,33.0,0.3,26.0,98.0,0.0,0.0,0.0,0.0,0.0


### For each Scenario, combine with baseline and save as input file¶

In [15]:
for ii in range (len(df.unstack(level=1))):
    SCEN = df.unstack(level=1).index[ii]
    SCEN=SCEN.replace('+', '_')
    filetitle = SCEN+'.csv'
    
    subtestfolder = os.path.join(testfolder, 'USA')
    if not os.path.exists(subtestfolder):
        os.makedirs(subtestfolder)
    filetitle = os.path.join(subtestfolder, filetitle)
    
    A = df.unstack(level=1).iloc[ii]

    A.name = 'new_Installed_Capacity_[MW]'
    A = pd.DataFrame(A)
    A.index=pd.PeriodIndex(A.index, freq='A')
    A = pd.DataFrame(A)
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 0.85 # marketshares['Si']
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] * 1000   # ReEDS file is in GW.
    # Add other columns
    A = pd.concat([A, baseline.reindex(A.index)], axis=1)
   
    header = "year,new_Installed_Capacity_[MW],mod_eff,mod_reliability_t50,mod_reliability_t90,"\
    "mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,"\
    "mod_Repair,mod_MerchantTail,mod_Reuse\n"\
    "year,MW,%,years,years,%,years,%,%,%,%,%,%\n"

    with open(filetitle, 'w', newline='') as ict:
    # Write the header lines, including the index variable for
    # the last one if you're letting Pandas produce that for you.
    # (see above).
        for line in header:
            ict.write(line)

        #    savedata.to_csv(ict, index=False)
        A.to_csv(ict, header=False)